### AlgoSB 2022: Intrinsic Disorder in Proteins
## 25/11 Practical session
# WASCO: A Wasserstein-based statistical tool to compare conformational ensembles of intrinsically disordered proteins


### Get ready to use WASCO

WASCO is available at [https://gitlab.laas.fr/moma/WASCO](https://gitlab.laas.fr/moma/WASCO), together with its installation instructions. Once you have installed WASCO, please add this notebook to the folder where the WASCO notebooks are installed. This can be done by opening the WASCO notebooks using the command ``wasco-notebooks``, and then by uploading this file clicking on the "upload" button at the top right corner. Then, open the uploaded file.

Finally, run the following commands in order to load WASCO functions here.

In [ ]:
import ipynb
from ipynb.fs.full.build_frames import *
from ipynb.fs.full.sample_independent_replicas import *
from ipynb.fs.full.multiframe_conversion import *
from ipynb.fs.full.wmatrix import *
from ipynb.fs.full.wvector import *
from ipynb.fs.full.graphical_representation import *
from ipynb.fs.full.plot_structures import *

import os
import time
import sys
import shutil
import warnings # Optional
#warnings.filterwarnings("ignore") # Optional
path_to_notebook = os.path.abspath(os.getcwd())
print(path_to_notebook) # Visualize the notebook directory

### This notebook

WASCO is implemented in order to compute everything automatically. It gets a pair of ensembles as input, some specifications given by the practitioner, performs all the calculations automatically and returns the output. Here, we <b>split</b> WASCO into several steps to illustrate the corresponding methodology and the influence of some of its functions (account for uncertainty,  definition of overall distance). 

For each ensemble, the user must provide the path to a folder containing the ensemble data. It must contain:
* One .xtc file per replica, together with a .pdb file with topology information **or**,
* One multiframe .pdb file per replica **or**,
* If ensembles are given as a list of .pdb files (one file per conformation), one **folder** per replica, containing one .pdb file per conformation each.

Here, we will reproduce the <b>step-by-step comparison</b> of the pair of ensembles:
* Histatin ensemble before and after filtering based on experimental SAXS data.

We first specify where the corresponding data is located and the number of threads (cores) to be used.

In [ ]:
ensemble_1_name = 'histatin_pool'
ensemble_2_name = 'histatin_filtered'

ensemble_1_path = "/".join([path_to_notebook,'Examples',ensemble_1_name])
ensemble_2_path = "/".join([path_to_notebook,'Examples',ensemble_2_name])

# Number of available cores
print(int(os.cpu_count()))

In [ ]:
# Number of cores to use
N_cores = 4

Finally, we must specify a directory where results must be saved:

In [ ]:
if not os.path.exists("/".join([os.path.abspath(os.path.join(ensemble_1_path, os.pardir)),"_".join(['results',ensemble_1_name,ensemble_2_name])])):
    os.mkdir("/".join([os.path.abspath(os.path.join(ensemble_1_path, os.pardir)),"_".join(['results',ensemble_1_name,ensemble_2_name])]))

results_path = "/".join([os.path.abspath(os.path.join(ensemble_1_path, os.pardir)),"_".join(['results',ensemble_1_name,ensemble_2_name])])
#results_path = # another path

## 1. Comparison of a pair of replicas
We will first compare only the first replicas of each ensemble.
### 1.1 Reference frame construction

Let $L$ be the sequence length. For a given replica:
    
* For each conformation:
    * For each sequence position $i = 1,\ldots,L$:
        * We get the values of $(\phi,\psi)$ angles,
        * We build the reference system at position $i$, using the coordinates of its CA, C and N atoms,
        * We compute all the relative position vectors of every other residue at position $j\neq i$.

In [ ]:
# For the first replica of the first ensemble (contains 5000 conformations)
define_frames(pdb_folder = "/".join([ensemble_1_path,'histatin_brute_run1']),
              num_cores = N_cores, prot_name = "_".join([ensemble_1_name,'run1']), save_to =  results_path,
              name_variable = 'ipynb.fs.full.build_frames')

# For the first replica of the second ensemble (contains 1000 conformations)
define_frames(pdb_folder = "/".join([ensemble_2_path,'histatin_net_run1']),
              num_cores = N_cores, prot_name = "_".join([ensemble_2_name,'run1']), save_to =  results_path,
              name_variable = 'ipynb.fs.full.build_frames')

For each replica, we obtain a pair of arrays:

* The sample of its **global structure**, given as an array of shape [Number of conformations, Number of position pairs, Number of covariates = 11], to which one can access to have conformation/relative position-specific information. 

* The sample of its **local structure**, given as an array of shape [Number of conformations, Sequence length, Number of covariates = 4]. 

Each array is saved to a .hdf5 file (recommended). .hdf5 files allow accessing to sub-arrays without loading on memory the entire array. This is needed for future computation of ensemble comparison matrices.

### 1.2 Visualize the computed samples

We can take a look at the computed global/local structures:

* The <b>global structures</b> are probability distributions supported on the three-dimensional euclidean space. Therefore, their samples are <b>point clouds</b> in such space:

In [ ]:
plot_global(pos_1 = 3, pos_2 = 20, path_1 = results_path, path_2 = results_path, name_1 = 'histatin_pool_run1', name_2 = 'histatin_filtered_run1')

* <b>Local structures</b> are probability distributions supported on the two-dimensional flat torus. Their samples are commonly represented through <b>Ramachandran maps</b>. To ease visual comparison, we can plot their kernel density estimates (here, ignoring periodicity):

In [ ]:
plot_local(pos = 3,path_1 = results_path, path_2 = results_path, name_1 = 'histatin_pool_run1', name_2 = 'histatin_filtered_run1')

### 1.3 Comparing global and local structures with Wasserstein distance

We need to create subdirectories to save the computed distances (we will need this when adding several replicas to the comparison):

In [ ]:
wmat_path = "/".join([results_path,"_".join(['wmatrices',ensemble_1_name,ensemble_2_name])])
wvec_path = "/".join([results_path,"_".join(['wvectors',ensemble_1_name,ensemble_2_name])])

if not os.path.exists(wmat_path):
    os.mkdir(wmat_path)
if not os.path.exists(wvec_path):
    os.mkdir(wvec_path)

#### Distances between global structures

In [ ]:
wmatrix = w_matrix(prot_1 = "_".join([ensemble_1_name,'run1']), prot_2 = "_".join([ensemble_2_name,'run1']) , N_centers = 2000, N_cores = N_cores, data_path = results_path, name_variable = 'ipynb.fs.full.wmatrix')

# The resulting matrix needs to be saved
np.save("/".join([wmat_path,"_".join(["_".join([ensemble_1_name,'run1']),"_".join([ensemble_2_name,'run1']),'wmatrix.npy'])]), wmatrix)

#### Distances between local structures

In [ ]:
wvector = w_vector(prot_1 = "_".join([ensemble_1_name,'run1']), prot_2 = "_".join([ensemble_2_name,'run1']) , N_centers = 2000, N_cores = N_cores, data_path = results_path, name_variable = 'ipynb.fs.full.wvector')

# The resulting vector needs to be saved
np.save("/".join([wvec_path,"_".join(["_".join([ensemble_1_name,'run1']),"_".join([ensemble_2_name,'run1']),'wvector.npy'])]), wvector)

### 1.4 Visualizing results through WASCO output

Results are returned through a matrix, depicting both scales' results: global distances are included in the lower triangle and local distances along the diagonal. As, for now, no independent replicas are provided, the color scales correspond to the **inter-ensemble distances** between each pair of distributions.

The entry $(i,j)$ of the matrix coresponds to the distance between the distributions of the relative positions $i,j$ (one distribution per ensemble). It measures how different is the relative position of residue $i$ with respect to $j$ when changing from one ensemble to the other. The entry $(i,i)$ corresponds to the distance between the $(\phi, \psi)$ distributions of the $i$-th residue (one distribution per ensemble). It measures how different is the $i$-th $(\phi, \psi)$ distribution when changing from one ensemble to the other.

In [ ]:
wmatrix_plot(prot_name_1 = "_".join([ensemble_1_name,'run1']), prot_name_2 = "_".join([ensemble_2_name,'run1']),
             wmatrix_path = wmat_path, 
             wvector_path = wvec_path)

We can observe how the overall global distance changes if we don't weight the individual distances according to the relative position of residues along the sequence, i.e. multiplying by an increasing function $w_{ij}=w(|i-j|)$ of $|i-j|$, satisfying $w(L-1)=1$ and $w(0)=0$.

In [ ]:
wmatrix_plot(prot_name_1 = "_".join([ensemble_1_name,'run1']), prot_name_2 = "_".join([ensemble_2_name,'run1']),
             wmatrix_path = wmat_path, 
             wvector_path = wvec_path,
             ponderate = False)

#### About the output

* The ouput shows more important **global differences** for residue pairs far away from each other in the sequence (far from the diagonal), and small differences for closer pairs (close to the diagonal). In particular, the biggest changes appear between relative positions of residues $\sim 5-7$ with respect to $\sim 20-23$.

We can take a look at the corresponding local structures. However, visual comparison of point clouds (specially when they have different sample sizes) is not very informative. In any case, looking at the samples is a complementary visualization of the structures after analyzing the matrix, which is the main output of the comparison method.

For the biggest differences:

In [ ]:
plot_global(pos_1 = 6, pos_2 = 22, path_1 = results_path, path_2 = results_path, name_1 = 'histatin_pool_run1', name_2 = 'histatin_filtered_run1')

For the smallest differences:

In [ ]:
plot_global(pos_1 = 6, pos_2 = 7, path_1 = results_path, path_2 = results_path, name_1 = 'histatin_pool_run1', name_2 = 'histatin_filtered_run1')

* The output shows more important **local differences** for residues 1, 3 and 19. When can look at the corresponding Ramachandran maps.

For the biggest differences:

In [ ]:
plot_local(pos = 3,path_1 = results_path, path_2 = results_path, name_1 = 'histatin_pool_run1', name_2 = 'histatin_filtered_run1')

   For the smallest differences:

In [ ]:
plot_local(pos = 9,path_1 = results_path, path_2 = results_path, name_1 = 'histatin_pool_run1', name_2 = 'histatin_filtered_run1')

## 2. Including 2 replicas per ensemble to take uncertainty into account

We first build the reference systems to get samples from global and local structures for the second replica of each ensemble.

In [ ]:
# For the second replica of the first ensemble (contains 5000 conformations)
define_frames(pdb_folder = "/".join([ensemble_1_path,'histatin_brute_run2']),
              num_cores = N_cores, prot_name = "_".join([ensemble_1_name,'run2']), save_to =  results_path,
              name_variable = 'ipynb.fs.full.build_frames')

# For the second replica of the second ensemble (contains 1000 conformations)
define_frames(pdb_folder = "/".join([ensemble_2_path,'histatin_net_run2']),
              num_cores = N_cores, prot_name = "_".join([ensemble_2_name,'run2']), save_to =  results_path,
              name_variable = 'ipynb.fs.full.build_frames')

Now, we are able to get a more precise estimate of the **inter-ensemble** distances if we compare the second replicas between them. Then, we will use the average of the inter-ensemble distances for the two replicas as an estimate of the *real* inter-ensemble difference between the pair of ensembles.

### 2.1 Improving the estimation of the inter-ensemble distances

#### Distances between inter-ensemble global structures (for the second replica)

In [ ]:
wmatrix = w_matrix(prot_1 = "_".join([ensemble_1_name,'run2']), prot_2 = "_".join([ensemble_2_name,'run2']) , N_centers = 2000, N_cores = N_cores, data_path = results_path, name_variable = 'ipynb.fs.full.wmatrix')

# The resulting matrix needs to be saved
np.save("/".join([wmat_path,"_".join(["_".join([ensemble_1_name,'run2']),"_".join([ensemble_2_name,'run2']),'wmatrix.npy'])]), wmatrix)

#### Distances between inter-ensemble local structures (for the second replica)

In [ ]:
wvector = w_vector(prot_1 = "_".join([ensemble_1_name,'run2']), prot_2 = "_".join([ensemble_2_name,'run2']) , N_centers = 2000, N_cores = N_cores, data_path = results_path, name_variable = 'ipynb.fs.full.wvector')

# The resulting vector needs to be saved
np.save("/".join([wvec_path,"_".join(["_".join([ensemble_1_name,'run2']),"_".join([ensemble_2_name,'run2']),'wvector.npy'])]), wvector)

### 2.2 Estimating uncertainty through intra-ensemble distances

We compute the intra-ensemble global and local distances to estimate the effect of uncertainty and integrate it to the inter-ensemble comparison. We first create two new directories to store the computed intra-ensemble distances.

In [ ]:
wmat_ind_path = "/".join([results_path,"_".join(['intra_wmatrices',ensemble_1_name,ensemble_2_name])])
wvec_ind_path = "/".join([results_path,"_".join(['intra_wvectors',ensemble_1_name,ensemble_2_name])])

if not os.path.exists(wmat_ind_path):
    os.mkdir(wmat_ind_path)
if not os.path.exists(wvec_ind_path):
    os.mkdir(wvec_ind_path)

And we can compute the global and local intra-ensemble distances i.e. between different replicas of the same ensemble:

#### Distances between intra-ensemble global structures

In [ ]:
wmatrix = w_matrix(prot_1 = "_".join([ensemble_1_name,'run1']), prot_2 = "_".join([ensemble_1_name,'run2']) , N_centers = 2000, N_cores = N_cores, data_path = results_path, name_variable = 'ipynb.fs.full.wmatrix')

# The resulting matrix needs to be saved
np.save("/".join([wmat_ind_path,"_".join(["_".join([ensemble_1_name,'run1']),"_".join([ensemble_1_name,'run2']),'wmatrix.npy'])]), wmatrix)

#### Distances between intra-ensembles local structures

In [ ]:
wvector = w_vector(prot_1 = "_".join([ensemble_2_name,'run1']), prot_2 = "_".join([ensemble_2_name,'run2']) , N_centers = 2000, N_cores = N_cores, data_path = results_path, name_variable = 'ipynb.fs.full.wvector')

# The resulting vector needs to be saved
np.save("/".join([wvec_ind_path,"_".join(["_".join([ensemble_2_name,'run1']),"_".join([ensemble_2_name,'run2']),'wvector.npy'])]), wvector)

### 2.3 Visualizing the results corrected for uncertainty

The corrected distances are defined as the difference between the average of all inter-ensemble distances and the average of all the intra-ensemble distances:
$$
\Delta W=(\overline{W_{inter}}-\overline{W_{intra}})_+
$$
where $(x)_+=x$ if $x>0$ and $(x)_+=0$ otherwise. 

To represent the results, we choose an interpretable scale which uses the estimated uncertainty as a reference to which compare $\Delta W$. The scale is defined as
$$
\frac{\Delta W}{\overline{W_{intra}}},
$$
which represents the proportion of the intra-ensemble distances that represents $\Delta W$. For example, if for a given coefficient the score is $2$, that would mean that the inter-ensemble distances exceed the uncertainty by twice the uncertainty, so the corrected distances are relevant when compared to the intra-ensemble ones. A score of $0.1$ would mean that the inter-ensemble distances exceed the uncertainty by "only" the $10\%$ of the uncertainty.

Let's plot the output of WASCO when taking the two replicas into account:

In [ ]:
wmatrix_plot(prot_name_1 = "_".join([ensemble_1_name,'run1']), prot_name_2 = "_".join([ensemble_2_name,'run1']),
             wmatrix_path = wmat_path, 
             wmatrix_ind_folder = wmat_ind_path,
             wvector_path = wvec_path,
             wvector_ind_folder = wvec_ind_path)

As you see, the matrix has significantly changed with respect to the single replicas comparison. The integration of uncertainty removes the effect of intra-ensemble differences propagating from the corner to the diagonal, which no longer appears. Now, we still see some differences in the lower triangle for residue pairs $\sim 6-8$ with respect to $\sim 20-23$. These differences must be taken into account, as their corresponding score is higher than $2-3$. 

However, the biggest differences appear now for residue pairs close to the diagonal. Moreover, we see that *the most important global differences appear for residues whose local differences are very significant*. In other words, the most relevant changes on local structures (residues 1,3,19) propagate through the lower triangle and yield important global changes for the surrounding amino-acids.

Finally, we can see the effect of uncertainty and weighting the matrix coefficient when computing the overall score, by modifying the parameters ``overall_uncertainty`` (whether to compute the global score from corrected distances) and ``ponderate`` (whether to multiply matrix coefficients by $w_{ij}$ to compute the global score).

In [ ]:
wmatrix_plot(prot_name_1 = "_".join([ensemble_1_name,'run1']), prot_name_2 = "_".join([ensemble_2_name,'run1']),
             wmatrix_path = wmat_path, 
             wmatrix_ind_folder = wmat_ind_path,
             wvector_path = wvec_path,
             wvector_ind_folder = wvec_ind_path,
             overall_uncertainty = False,
             ponderate = False)

## 3. Automatic computation using the "real" WASCO

If you have some time, you can repeat all the computation but using the [comparison_tool](https://gitlab.laas.fr/moma/WASCO/-/blob/master/wasco/comparison_tool.ipynb) provided by WASCO. The tool takes one directory per ensemble as input, automatically identify the data formats and the several replicas, checks some considerations with the user and proceeds to compute the matrix automatically. It can be done through an interactive version, which only takes directories as inputs and checks the information with the user, or through a non-interactive version (suitable for launching long computations in remote servers), which takes all the required information as input (this requires a small understanding of what is happening by the user). 

We can take a look at the interactive version, which should return the same output as before:

In [ ]:
from ipynb.fs.defs.comparison_tool import comparison_tool

In [ ]:
comparison_tool(ensemble_1_path = ensemble_1_path,
                ensemble_1_name = 'histatin_pool', 
                ensemble_2_path = ensemble_2_path,
                ensemble_2_name = 'histatin_filtered', 
                results_path = results_path)